In [1]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    # if "2.Final_output" in list_of_folders:
    #     subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    # os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [5]:
import os
from Bio import SeqIO
import subprocess

family_group_list = ["3.Satyrine","4.Pierinae","5.Coliadinae","6.Heliconiinae_Danainae_Nymphalinae","7.Papilionidae","8.Lycaenidae"]
for family_group in family_group_list:
    output_location = f"/mnt/g/My Drive/Circadian Rhythm Genes Project/12.Cryptochrome_1a_Exon_Analysis/{family_group}"
    make_output_folder(output_location)
    genome_location = "/mnt/f/Genomes_2023-12-26"
    list_of_species = os.listdir(f"{output_location}/1.Blast_result")
    if 'desktop.ini' in list_of_species:
        list_of_species.remove("desktop.ini")
    # print(list_of_species)
    # list_of_species = ["Heliconius_charithonia"]
    # list_of_species = ["Maniola_hyperantus"]
    error_exons = ''
    for species in list_of_species:
        print(species)
        genome_file = get_genome_file(genome_location,species)
        print(genome_file)
        previous_coordinate_start = ''
        previous_scaffold = ''
        with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
            coordinate_file_list = coordinate_final_file.readlines()
        
        for i in range(1,len(coordinate_file_list)):
            
            coordinate_line_split = coordinate_file_list[i].split(",")
            
                
    #         print(coodinate_line_split[5])
            if coordinate_line_split[5] == "N":
                if previous_coordinate_start == '':
                    previous_coordinate_start = coordinate_line_split[2]
                    previous_scaffold = coordinate_line_split[1]
                complement = coordinate_line_split[4]
                start,stop = coordinate_line_split[2],coordinate_line_split[3]
                print(start,stop)
                query_name = coordinate_line_split[6]
                scaffold = coordinate_line_split[1]
                exon = f"Exon_{query_name.split('_')[-1]}"
                left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
                print(left_oh, right_oh)
    #             print(complement)
                if complement == "0":
                    if start < previous_coordinate_start or scaffold != previous_scaffold:
                        print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                        prompt = input("Continue (Y/N)?")
                        if prompt[0].lower() == "n":
                            assert False
                elif complement == "1":
                    if start > previous_coordinate_start or scaffold != previous_scaffold:
                        print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                        prompt = input("Continue (Y/N)?")
                        if prompt[0].lower() == "n":
                            assert False
                else:
                    print("ERROR")
                
            elif coordinate_line_split[5] == "Y":
                query_name = coordinate_line_split[6]
                exon = f"Exon_{query_name.split('_')[-1]}"
                scaffold = coordinate_line_split[1]
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                continue
                
            gene_sequence = get_gene_sequence(genome_location, 
                                              species, 
                                              genome_file, 
                                              scaffold,
                                              start,
                                              stop,
                                              complement,
                                              output_location)
            
            gene_sequence_2 = get_gene_sequence(genome_location, 
                                              species, 
                                              genome_file, 
                                              scaffold,
                                              int(start)-2,
                                              int(stop)+2,
                                              complement,
                                              output_location)
        
            current_exon = query_name.split('_')[-1]
            if current_exon != "1":        
                if gene_sequence_2[:2].lower() != "ag":
                    print(species, current_exon)
                    print(gene_sequence)
                    print(gene_sequence_2)
                    print("Not AG")
                    test = input("quit?")
                    if test == "y":
                        assert False
                    # assert False
            print(current_exon, gene_sequence[-3:].translate())
            if gene_sequence[-3:].translate() != "*":
                
                # print(gene_sequence_2[-2:].lower())
                if gene_sequence_2[-2:].lower() != "gt":
                    print(current_exon)
                    print(len(coordinate_file_list))
                    print(species, current_exon)
                    print(gene_sequence)
                    print(gene_sequence_2)
                    print("Not gt")
                    test = input("quit?")
                    if test == "y":
                        assert False
            if current_exon == "1":
                if complement == 1:
                    translated = gene_sequence.reverse_complement.translate()
                else:
                    translated = gene_sequence.translate()
                print(translated)
                if "*" in translated:
                    error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                else:
                    exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                    print(exon_output)
            
                    write_exon_output(output_location,species,exon,exon_output)
            elif current_exon == "16":
                if complement == 1:
                    translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                    print(gene_sequence)
                else:
                    translated = gene_sequence[int(left_oh):].translate()
                    print(gene_sequence)
    
                if "*" in translated[:-1]:
                    error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                    print(translated)
                else:
                    
                    exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                    print(exon_output)
                    write_exon_output(output_location,species,exon,exon_output)
    #         assert False
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
    
                write_exon_output(output_location,species,exon,exon_output)
            
    #         assert False
            previous_coordinate_start = start
            previous_scaffold = scaffold
        error_exons += "\n\n"
    #     print(coordinate_file_list)
    #     break
    coordinate_file = ''
    print(error_exons)
    with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
        error_out_file.write(error_exons)

Bicyclus_anynana
GCF_947172395.1_ilBicAnyn1.1_genomic.fna
14967419 14967548
0 1
Getting Gene
Getting Gene
1 L
MAGGSVLWFRHGLRLHDNPSLNDALEDKAAPFFPMFIFDGETA
>Bicyclus_anynana_Exon_1_NC_069093.1_14967419_14967548_left_0_right_1
ATGGCCGGCGGCAGCGTGCTGTGGTTCCGTCACGGGTTGCGATTGCACGACAACCCATCTTTGAACGACGCCTTGGAAGACAAGGCCGCTCCCTTCTTCCCGATGTTCATTTTCGACGGAGAAACTGCTG
14972782 14972928
2 1
Getting Gene
Getting Gene
2 L
>Bicyclus_anynana_Exon_2_NC_069093.1_14972782_14972928_left_2_right_1
GAACCAAACTGGTTGGCTACAACCGCATGCGCTATCTTCTGGAGGCGCTCAACGACCTGGACAACCAGTTCAGGAAGTATGGCGGCCGGCTGCATATGATCAAGGGAAAACCTGACGTTGTGTTCAAACGATTGTGGAAAGAATTTG
14973669 14973856
2 0
Getting Gene
Getting Gene
3 L
>Bicyclus_anynana_Exon_3_NC_069093.1_14973669_14973856_left_2_right_0
GTATCCGTAAGATATGTTTCGAGCAAGACTGCGAGCCGATCTGGCGCATGCGCGACGAGAGCGTGCGCGGGCTGTGCCGCGAGCTGGGCGTGGCGTGTCGCGAGAGCGTGTCGCACACGCTGTGGGACCCCGACACCGTCATACGCGCCAACGGTGGCATCCCGCCCCTCACTTATCAGATGTTTCTG
14974731 14974847
0 0
Getting Gene
Getting Gene
4 K
>Bicyclus_an

Continue (Y/N)? y


Getting Gene
Getting Gene
9 L
>Eurema_hecabe_Exon_9_JADANM010006710.1_97104_97286_left_1_right_2
aTGGAAGGAGGGCCTGACGGGGTTCCCGTTCGTGGACGCGGCGATGCGGCAGCTGCGCGCGGAGGGCTGGCTGCACCACGCGCTGCGCAACGCCGTCGCCTCCTTCCTCACGCGCGGCACGCTCTGGCTCTCCTGGGAGCACGGCCTGAAGCACTTCCTGCGCTACCTGCTCGATGCCGACTG
96445 96612
1 2
Getting Gene
Getting Gene
10 H
>Eurema_hecabe_Exon_10_JADANM010006710.1_96445_96612_left_1_right_2
GTCTGTGTGTGCGGGCAACTGGATGTGGGTGTCGTCGAGCGCGTTCGAGGCGCTGCTGGACTCGGGCGAGTGCGCGTGCCCCGTGCGGCTCGGCCAGCGCCTCGAGCCCAGCGGCCGCTACGTGCGCCGCTACGTGCCCGAGCTGGCGCACGTGCCCACCGAGTACAT
96135 96258
1 0
Getting Gene
Getting Gene
11 Q
>Eurema_hecabe_Exon_11_JADANM010006710.1_96135_96258_left_1_right_0
aTACGAGCCGTGGAAGGCGCCGTTGGAAGTCCAGATTCGCGCGAACTGCGTGATCGGGCGCGACTACCCGCCGCCCATCGTCGATCATCTCGCCGCTGCCAGGAGTAACCGCAGCGCTATGCAG
95336 95452
0 0
Getting Gene
Getting Gene
12 *
>Eurema_hecabe_Exon_12_JADANM010006710.1_95336_95452_left_0_right_0
GAATTACGTCGAATCCTAGATAAGGCTCAGCCGCATTGCTGCCCCTCGTCGGAGGAAGAAATTCGTCAGTTCATGTGGCT

quit? 


>Leptidea_sinapis_Exon_2_NC_066312.1_35046475_35046621_left_2_right_1
GTACTAGTCTGGTTGGATATAACCGCATGCGATATCTACTGGAAGCTCTTGACGACCTGGGCTCACAGCTTAGACAACATGGAGGTCGACTCCACCTGGTGAAGGGAAACCCAACTGATATCATCCGCCGTCTTCGGGAAGAGTTGg
35042891 35043078
2 0
Getting Gene
Getting Gene
3 L
>Leptidea_sinapis_Exon_3_NC_066312.1_35042891_35043078_left_2_right_0
GTCTGAAGAAGATATGCTTCGAGCAAGACTGCGAGCCGATCTGGCGCGCGCGAGACGACCGCGTGCGCAACCTGTGCCGCGAGGTGGGCGTGGTGTGTCGCGAGCACGTGTCGCACACCCTGTGGGAGCCTGACACCATCCTGCGGCACAACGGGAACATACCGCCGCTCACCTACCAGATGTTCTTG
35041650 35041766
0 0
Getting Gene
Getting Gene
4 K
>Leptidea_sinapis_Exon_4_NC_066312.1_35041650_35041766_left_0_right_0
cacaCCGTGTCAATAATCGGCGACCCGCCCCGGCCCGTCAGCGACGTGGACTTGCGGGAAGTACAGTTCGGTGCACTACCCGACGCGTTCTGCAAAGAATTTTGTGTCTTTGATAAA
35040819 35040952
0 2
Getting Gene
Getting Gene
5 Q
>Leptidea_sinapis_Exon_5_NC_066312.1_35040819_35040952_left_0_right_2
ACTCCAAAGCCAGAGGACTTGGGCGTGTTTCTGGAGAACGAAGACATCCGCATGATCAGGTGGGTGGGCGGCGAGACCGCGGCGCTCAAGCAGATGGAGCAGCGGCTGG

...

>Mechanitis_mazaeus_Exon_5_OY365748.1_5927909_5928042_left_0_right_2
gtACCGAAGCCAGAAGACCTAGGTGTATTTCTGGAGAACGAAGATATCCGCATGATTCGGTGGGTGGGGGGAGAGACTGTGGCTTTGAAGCAGATGCAGGAACGCCTCGCTGTGGAGTACGAGACCTTCTGCAG
5927633 5927728
1 2
Getting Gene
Getting Gene
6 S
>Mechanitis_mazaeus_Exon_6_OY365748.1_5927633_5927728_left_1_right_2
AGGTTCCTACTTACCAACCCACGGGAACCCCGATCTACTGGGACCCCCAATATCCTTAAGTCCCGCTTTGCGATTTGGATGTCTGTCTGTGCGTCG
5927493 5927566
1 1
Getting Gene
Getting Gene
7 L
>Mechanitis_mazaeus_Exon_7_OY365748.1_5927493_5927566_left_1_right_1
tttctacTGGAGTGTTCAAGACCTGTTCCAGCAGGTACACCAAGGTCGTCTCGCTTCCACACAGTTTATTActg
5927004 5927130
2 2
Getting Gene
Getting Gene
8 E
>Mechanitis_mazaeus_Exon_8_OY365748.1_5927004_5927130_left_2_right_2
GTCAGTTGATTTGGCGCGAGTATTTCTACACTATGAGTGTTAATAACCCCAACTATGCCCAAATGGCTGACAACCCTATTTGCTTAGACATCCCGTGGAAGGAGCCAGAGAATGATGAGCTTCAgag
5926280 5926462
1 2
Getting Gene
Getting Gene
9 L
>Mechanitis_mazaeus_Exon_9_OY365748.1_5926280_5926462_left_1_right_2
ATGGAAGGAGGGTCGCACGG

quit? 


>Mechanitis_messenoides_Exon_5_OY365760.1_15056276_15056409_left_0_right_2
gtaCCGAAGCCAGAAGACCTAGGTGTATTTCTGGAGAACGAAGATATCCGCATGATTCGGTGGGTGGGGGGAGAGACTGTGGCTTTGAAGCAGATGCAGGAACGCCTCGCTGTGGAGTACGAGACCTTCTGCAG
15056590 15056685
1 2
Getting Gene
Getting Gene
6 S
>Mechanitis_messenoides_Exon_6_OY365760.1_15056590_15056685_left_1_right_2
AGGTTCCTACTTACCAACACACGGGAACCCCGATCTACTGGGACCCCCAATATCCTTAAGTCCCGCTTTGCGATTTGGATGTCTGTCTGTGCGTcg
15056755 15056828
1 1
Getting Gene
Getting Gene
7 L
>Mechanitis_messenoides_Exon_7_OY365760.1_15056755_15056828_left_1_right_1
ATTCTACTGGAGTGTTCAAGACCTGTTCCAGCAGGTACACCAAGGTCGTCTCGCTTCTACACAGTTTATTACTG
15057088 15057214
2 2
Getting Gene
Getting Gene
8 E
>Mechanitis_messenoides_Exon_8_OY365760.1_15057088_15057214_left_2_right_2
GTCAGTTGATTTGGCGCGAGTATTTCTACACTATGAGTGTTAATAACCCCAACTATGCCCAAATGGCTGACAACCCTATTTGCTTAGACATCCCGTGGAAGGAGCCAGAGAATGATGAGCTTCAgag
15057852 15058034
1 2
Getting Gene
Getting Gene
9 L
>Mechanitis_messenoides_Exon_9_OY365760.1_15057852_150580

quit? 


>Iphiclides_podalirius_Exon_7_OW152822.1_2871054_2871127_left_1_right_1
ATTCTACTGGGCTCTTCAAGATTTATTCAACCAGGTTCACCAAGGACGCATTCCCGCCACCCAGTTGGTAACAG
2871201 2871327
2 2
Getting Gene
Getting Gene
8 K
>Iphiclides_podalirius_Exon_8_OW152822.1_2871201_2871327_left_2_right_2
GTCAACTGATATGGCGAGAGTATTTCTACACTATGAGCGTGAACAATCCGAACTACGGACAAATGGCTGGGAATCCCATTTGTCTCGATATACCATGGAAGAATCCCGAAGGTGACGAGCTGAAAAG
2871960 2872142
1 2
Getting Gene
Getting Gene
9 L
>Iphiclides_podalirius_Exon_9_OW152822.1_2871960_2872142_left_1_right_2
ATGGAAGGAGGGCCGCACCGGCTTCCCGTTCATCGACGCGGCGATGCGGCAGCTCCGCTCTGAGGGTTGGCTGCACCACGCCCTGAGGAACACCGTCGCCTCCTTCCTCACGCGGGGAACGCTCTGGCTCTCTTGGGAGCACGGCCTGGAGCATTTCCTCAAGTACCTTCTGGATGCCGACTG
2872962 2873129
1 2
Getting Gene
Getting Gene
10 H
>Iphiclides_podalirius_Exon_10_OW152822.1_2872962_2873129_left_1_right_2
GTCGGTGTGCGCCGGCAACTGGATGTGGGTGTCGTCGAGCGCGTTCGAGGCGCTGCTGGACTCGAGCGAGTGCGCGTGCCCAGTGCTGCTGGGGCGTAGGCTCGAGCCCAGTGGCGAGTACGTGCGCCGCTACGTGCCCGAGCTGGCGCGCATGCCCGACAACTACAT
2873